In [ ]:
%run sources/sequence_dataset.py
%run sources/sequence_walking_tree.py
%run sources/sequence_walking_forest.py


SAMPLES = 140

In [ ]:
"""
from networkx.drawing.nx_pydot import to_pydot
from IPython.display import Image, display
import networkx as nx

def generate_tree(g, t, k=0):
    newline = "\n"
    g.add_node(k, label=f'{t["test_type"] if "test_type" in t else ""}{t["test"] if "test" in t else ""}{newline + "true_samples - "+str(t["true_samples"])+ " " if "true_samples" in t else "" }{newline+"false_samples - "+str(t["false_samples"]) if "false_samples" in t else "" }{newline+str(t["probabilities"])}'  )
    next_k = k + 1
    for key in [True, False]:
        if key in t:
            current_k, next_k = generate_tree(g, t[key], next_k)
            g.add_edge(k, current_k, label= f'{key}')
    return k, next_k

for i,t in enumerate(F.trees):
    G = nx.DiGraph()
    generate_tree(G,t.tree)
    pydot_graph = to_pydot(G)
    dot_file_path = f"tree{i}.dot"
    pydot_graph.write_dot(dot_file_path)
    !dot -Tpng tree{i}.dot -o tree{i}.png
    #display(Image(f"tree{i}.png"))
"""   

In [ ]:
def read_event_logs(csv_path):
    r = pd.read_csv(csv_path)
    return list(r["sequence"].apply(lambda x: eval(x) )), list(r["class"])

X,Y = read_event_logs("datasets/20240124_1525_fitbit_p02_quartiles.csv")

DRIFT = 30
PREDICT = 10

Z = SequenceDataset()
Z.fit(X[SAMPLES - DRIFT:SAMPLES],Y[SAMPLES - DRIFT:SAMPLES])

Z.sample_df.loc[DRIFT - PREDICT:DRIFT,0 ] = 0

In [ ]:
Z.predictions = {}

from math import log2
mh = int(log2(Z.last_position_df.last_position.mean()))
ms = int(log2(len(Z.raw_x)))
rs = int(log2(max([len(l) for l in Z.raw_x])))
mh, ms, rs

In [ ]:
F = SequenceWalkingForest(number_of_trees=100, tree_parameters={"max_height": 4  })
F.fit(Z)

In [ ]:
F.predict(Z)